# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy as np
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                # skip first line that is a header
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

# define a data fram with two columns
data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [2]:
data.head()

,class,message
/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a,spam,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam/00373.ebe8670ac56b04125c25100a36ab0510,spam,ATTENTION: This is a MUST for ALL Computer Use...
/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216,spam,This is a multi-part message in MIME format.\n...
/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam/00210.050ffd105bd4e006771ee63cabc59978,spam,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
/Users/jeromeboyer/Code/jbcodeforce/ml-basics/DataScience-Python3/emails/spam/00033.9babb58d9298daa2963d4f514193d7d6,spam,This is the bottom line. If you can GIVE AWAY...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [8]:
# extract words and count the occurance of the words in each individual message
# Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

# Multinomial naive bayes
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [9]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'],
      dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [21]:
classifier = MultinomialNB()
# Separate data into train and test sets
splitIndex = np.random.rand(len(data)) < 0.8
train = data[splitIndex]
train.head()
tests = data[~splitIndex]
tests.head()
# Do a new model based on the training set 
train_counts = vectorizer.fit_transform(train['message'].values)

['00', '000', '0000', '000000', '000000000', '000000000000000000000', '00000254', '000033', '000058', '000066', '000071', '000080', '00008b', '000099', '0000a0', '0000bf', '0000c4', '0000cc', '0000ek', '0000ff', '0001015', '0001123', '000184', '000185', '000187', '000188', '000191', '000193', '000194', '000197', '000198', '0001pt', '0001zd', '000201', '000202', '000203', '000204', '000205', '000206', '000208', '000209', '000210', '000213', '000214', '000218', '000219', '000221', '000222', '000224', '000225', '000226', '000228', '000229', '000230', '000231', '000290', '0003', '000393', '0003wq', '000402', '000426', '0004es', '000529', '000541', '000544', '000564', '000568', '000581', '000609', '000613', '000614', '000615', '000619', '000625', '000637', '000640', '000641', '000644', '000647', '000653', '000707', '0007b1d4', '0008', '00085k', '000e', '000s', '000th', '001', '0011k2hgsddos121zxn', '0013255', '001387', '001484', '0017', '001854', '001oyhw0942jqsk5', '001zrgsjtxqsod7antwgmpn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [22]:
classifier.fit(train_counts, train['class'].values)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
# Assess prediction on test set
test_counts = vectorizer.transform(tests)

predictions = classifier.predict(test_counts)

In [25]:
predictions

array(['spam', 'ham'],
      dtype='<U4')